In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
gt = pd.read_csv('datasets\\Global_terrorist.csv', encoding='ISO-8859-1')


In [8]:
gt.columns

Index(['eventid', 'iyear', 'imonth', 'iday', 'approxdate', 'extended',
       'resolution', 'country', 'country_txt', 'region',
       ...
       'addnotes', 'scite1', 'scite2', 'scite3', 'dbsource', 'INT_LOG',
       'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related'],
      dtype='object', length=135)

<h1> Filtering Data </h1>

In [5]:
filter1 = gt[gt['region_txt'] == 'Middle East & North Africa'] #50k

In [6]:
filter2 = filter1[filter1['attacktype1_txt'] == 'Bombing/Explosion'] #30k

In [7]:
filter3 = filter2[filter2['nkill'] >= 10] #1,800

In [10]:
filter4 = filter3[filter3['iyear'] >= 2000] #1737

<h1> Data Aggregation </h1>

In [11]:
ME_terror = filter4.groupby(['longitude','latitude']).size().reset_index(name = 'incident_count')

<h1> Visualization of GeoSpatial using FOLIUM </h1>

<h1>Regional Analysis </h1>

In [12]:
import folium 
from folium.plugins import HeatMap

In [13]:
heat_gt = ME_terror[['latitude','longitude', 'incident_count']].values.tolist()

In [14]:
my_map = folium.Map(location = [30,40], zoom_start = 4)

In [15]:
HeatMap(heat_gt).add_to(my_map)

In [16]:
my_map

<h1>Regional Analysis: Distribution of Terrorism Events Across Southeast Asia (2012 to Present)</h1>

In [19]:
filtered_gt_south_east = gt[(gt['region_txt'] == 'Southeast Asia') & (gt['iyear'] >= 2012)]

print(f'Filtered dataset size: {len(filtered_gt_south_east)} rows')

aggregated_gt_south_east = filtered_gt_south_east.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

def normalizer(gt, column):
    return (gt[column] - gt[column].min()) / (gt[column].max() - gt[column].min())

aggregated_gt_south_east['normalized_weight'] = normalizer(aggregated_gt_south_east, 'incident_count')

heat_data = aggregated_gt_south_east[['latitude', 'longitude', 'normalized_weight']].values.tolist()

m = folium.Map(location=[8, 110], zoom_start=5.5)

HeatMap(heat_data).add_to(m)

m

Filtered dataset size: 6026 rows


In [25]:
filtered_gt_south_east['country_txt'].value_counts()

country_txt
Philippines    3542
Thailand       1960
Myanmar         276
Indonesia       181
Malaysia         55
Laos              8
Cambodia          2
Vietnam           2
Name: count, dtype: int64

<h1>Crime Analysis: Kidnapping and Hostage-Taking in the European Region (Overall)</h1>

In [23]:
filtered_gt_european = gt[(gt['region_txt'] == 'Western Europe') | (gt['region_txt'] == 'Eastern Europe')]
filtered_gt_european = filtered_gt_european[filtered_gt_european['iyear'] >= 2000]

filtered_gt_european = filtered_gt_european[(filtered_gt_european['attacktype1_txt'] == 'Hostage Taking (Kidnapping)') | (filtered_gt_european['attacktype1_txt'] == 'Hostage Taking (Barricade Incident)')]


print(f'filtered dataset size: {len(filtered_gt_european)} rows')



aggregated_gt_european = filtered_gt_european.groupby(['latitude','longitude']).size().reset_index(name='incident_count')



aggregated_gt_european['normalized_weight'] = normalizer(aggregated_gt_european,'incident_count')

heat_data = aggregated_gt_european[['latitude','longitude','normalized_weight']].values.tolist()

m  = folium.Map(location=[50,20], zoom_start=4)

HeatMap(heat_data).add_to(m)

m



filtered dataset size: 205 rows


In [24]:
filtered_gt_european['country_txt'].value_counts()

country_txt
Ukraine           129
Russia             31
France             13
Macedonia          11
United Kingdom      8
Greece              3
Yugoslavia          2
Kosovo              2
Germany             2
Italy               2
Spain               1
Albania             1
Name: count, dtype: int64

<h1>Casualty Analysis: Middle East & North Africa - Bombing and Explosion Incidents with a Minimum of 40 Deaths</h1>

In [29]:
filtered_gt_midd_east = gt[
    (gt['region_txt'] == 'Middle East & North Africa') & 
    (gt['iyear'] >= 2000) & 
    (gt['attacktype1_txt'] == 'Armed Assault') & 
    (gt['targtype1_txt'].isin(['Government (General)', 'Transportation', 'Financial']))
]

print(f'Filtered dataset size: {len(filtered_gt_midd_east)} rows')


aggregated_gt_mid_east = filtered_gt_midd_east.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

def normalizer(gt, column):
    return (gt[column] - gt[column].min()) / (gt[column].max() - gt[column].min())

aggregated_gt_mid_east['normalized_weight'] = normalizer(aggregated_gt_mid_east, 'incident_count')

heat_data = aggregated_gt_mid_east[['latitude', 'longitude', 'normalized_weight']].values.tolist()

m = folium.Map(location=[35, 40], zoom_start=5)

HeatMap(heat_data).add_to(m)

m

Filtered dataset size: 794 rows


In [30]:
filtered_gt_midd_east['country_txt'].value_counts().reset_index()

,country_txt,count
0,Iraq,523
1,Yemen,85
2,Libya,54
3,West Bank and Gaza Strip,45
4,Turkey,23
5,Algeria,16
6,Egypt,14
7,Israel,8
8,Lebanon,8
9,Iran,7


<h1>Demographic Analysis: Distribution of Civilian Casualties in Public Spaces Worldwide (2016).</h1>

In [35]:
filtered_gt_civilian_attk = gt[
    (gt['iyear'] == 2016) & 
    (gt['targtype1_txt'].isin(['Markets', 'Public Park', 'Tourists'])) & 
    (gt['nkill'] >= 10)
]

print(f"Filtered dataset size: {len(filtered_gt_civilian_attk)} rows")

aggregated_gt_civilian_attk = filtered_gt_civilian_attk.groupby(['latitude', 'longitude']).size().reset_index(name='incident_count')

def normalizer(gt, column):
    return (gt[column] - gt[column].min()) / (gt[column].max() - gt[column].min()) if gt[column].max() != gt[column].min() else gt[column]

aggregated_gt_civilian_attk['normalized_weight'] = normalizer(aggregated_gt_civilian_attk, 'incident_count')

if 'normalized_weight' in aggregated_gt_civilian_attk.columns:
    aggregated_gt_civilian_attk = aggregated_gt_civilian_attk.dropna(subset=['latitude', 'longitude', 'normalized_weight'])
else:
    print("Column 'normalized_weight' is missing—check normalizer() application.")

heat_data = aggregated_gt_civilian_attk[['latitude', 'longitude', 'normalized_weight']].values.tolist()

m = folium.Map(location=[0, 0], zoom_start=2)

HeatMap(heat_data).add_to(m)

m

Filtered dataset size: 2 rows


<h1>Target Analysis: Educational Target Type in the Middle East and North Africa Region</h1>

In [38]:
filtered_gt_education = gt[gt['region_txt'] == 'Middle East & North Africa']

filtered_gt_education = filtered_gt_education[filtered_gt_education['iyear'] <= 2000]

filtered_gt_education = filtered_gt_education[filtered_gt_education['targtype1_txt'] == 'Educational Institution']


print(f'filtered dataset size: {len(filtered_gt_education)} rows')



aggregated_gt_edu = filtered_gt_education.groupby(['latitude','longitude']).size().reset_index(name='incident_count')



aggregated_gt_edu['normalized_weight'] = normalizer(aggregated_gt_edu,'incident_count')

heat_data = aggregated_gt_edu[['latitude','longitude','normalized_weight']].values.tolist()

m  = folium.Map(location=[30,40], zoom_start=4)

HeatMap(heat_data).add_to(m)

m

filtered dataset size: 244 rows


In [39]:
filtered_gt_education['country_txt'].value_counts().reset_index()

,country_txt,count
0,Turkey,83
1,Algeria,62
2,Lebanon,44
3,West Bank and Gaza Strip,17
4,Israel,15
5,Iran,10
6,Syria,4
7,Jordan,2
8,Egypt,2
9,Morocco,2
